<td>   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a></td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/basics/quick_start.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/basics/quick_start.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Quick Start

This notebook is intended to be a quick overview on Labelbox-Python SDK by demonstrating a simple but common work flow.

In this guide, we will be:

1. Creating a dataset and importing a image data row
2. Creating a ontology
3. Creating a project and attaching our ontology
4. Sending our data row to our project by creating a batch
5. Exporting from our project

This notebook is geared towards new users of our SDK.

## Setup

We first need to install the labelbox library and then import the SDK module. It is recommended to install "labelbox[data]" over labelbox to obtain all the correct dependencies. We will also be importing the Python UUID library to generate unique IDs for the variety of objects that will be created with this notebook.

In [ ]:
%pip install -q "labelbox[data]"

In [ ]:
import labelbox as lb
import uuid

## API Key and Client
Provide a valid API key below to connect to the Labelbox client properly. For more information, please review the [Create API Key](https://docs.labelbox.com/reference/create-api-key) guide.

In [ ]:
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Step 1: Create Dataset and Import Data Row

Below, we will create a dataset and then attach a publicly hosted image data row. Typically, you would either import data rows hosted on a cloud provider (_recommended_) or import them locally. For more information, visit our [import image data section](https://docs.labelbox.com/reference/image) in our developer guides.

- Data rows are internal representations of an asset in Labelbox. A data row contains the asset to be labeled and all of the relevant information about that asset
- A dataset is a collection of data rows imported into Labelbox. They live inside the [_Catalog_](https://docs.labelbox.com/docs/catalog-overview) section of Labelbox.

In [ ]:
# Create dataset from client
dataset = client.create_dataset(name="Quick Start Example Dataset")

global_key = str(uuid.uuid4())  # Unique user specified ID

# Data row structure
image_data_rows = [{
    "row_data":
        "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key":
        global_key,
    "media_type":
        "IMAGE",
}]

# Bulk import data row
task = dataset.create_data_rows(image_data_rows)  # List of data rows
task.wait_till_done()
print(task.errors)  # Print any errors

## Step 2: Creating an Ontology

Before we send our data row to a labeling project we first must create an ontology. In the example below we will be creating a simple ontology with a bounding box tool and a checklist classification feature. For more information, visit the [ontology section](https://docs.labelbox.com/reference/ontology) inside our developer guides. 

* An ontology is a collection of annotations and their relationships (also known as a taxonomy). Ontologies can be reused across different projects. It is essential for data labeling, model training, and evaluation. Created ontologies with there associated features are located inside the _Schema_ section within Labelbox.

In [ ]:
# Bounding box feature
object_features = [
    lb.Tool(
        tool=lb.Tool.Type.BBOX,
        name="regulatory-sign",
        color="#ff0000",
    )
]

# Checklist feature
classification_features = [
    lb.Classification(
        class_type=lb.Classification.Type.CHECKLIST,
        name="Quality Issues",
        options=[
            lb.Option(value="blurry", label="Blurry"),
            lb.Option(value="distorted", label="Distorted"),
        ],
    )
]

# Builder function
ontology_builder = lb.OntologyBuilder(tools=object_features,
                                      classifications=classification_features)

# Create ontology
ontology = client.create_ontology(
    "Ontology from new features",
    ontology_builder.asdict(),
    media_type=lb.MediaType.Image,
)

## Step 3: Creating a Project and Attaching our Ontology

Now that we have made our ontology, we are ready to create a project where we can label our data row.

* Projects are labeling environments in Labelbox similar to a factory assembly line for producing annotations. The initial state of the project can start with raw data, pre-existing ground truth, or pre-labeled data.

In [ ]:
# Create a new project
project = client.create_project(
    name="Quick Start Example Project",
    media_type=lb.MediaType.Image,  # specify the media type
)

# Attach created ontology
project.setup_editor(ontology)

## Step 4: Sending our Data Row to our Project by Creating a Batch

With our project created, we can send our data rows by creating a batch. Our data rows will start in the initial labeling queue, where labelers are able to annotate our data row.

* A batch is a curated selection of data rows you can send to a project for labeling. You can create a batch with a combination of data rows within any dataset. For more information on creating batches, review the [batches section](https://docs.labelbox.com/reference/batch#create-a-batch) of our developer guides.

In [ ]:
project.create_batch(
    name="Quick Start Example Batch" + str(uuid.uuid4()),
    global_keys=[
        global_key
    ],  # Global key we used earlier in this guide to create our dataset
    priority=5,
)

## Step 5: Exporting from our Project

We have now successfully set up a project for labeling using only the SDK! 🚀 

From here, you can either label our data row directly inside the [labeling queue](https://docs.labelbox.com/docs/labeling-queue) or [import annotations](https://docs.labelbox.com/reference/import-image-annotations) directly through our SDK. Below we will demonstrate the final step of this guide by exporting from our project. Since we did not label any data rows or import annotations within this guide, no labels will be presented on our data row. For a full overview of exporting, visit our [export overview](https://docs.labelbox.com/reference/label-export) developer guide.

In [ ]:
# Start export from project
export_task = project.export()
export_task.wait_till_done()

# Conditional if task has errors
if export_task.has_errors():
    export_task.get_buffered_stream(stream_type=lb.StreamType.ERRORS).start(
        stream_handler=lambda error: print(error))

# Start export stream
stream = export_task.get_buffered_stream()

# Iterate through data rows
for data_row in stream:
    print(data_row.json)

## Clean up

This section serves as an optional clean-up step to delete the Labelbox assets created within this guide. You will need to uncomment the delete methods shown.

In [ ]:
# project.delete()
# client.delete_unused_ontology(ontology.uid)
# dataset.delete()